In [1]:
from __future__ import print_function

## Exercise 2: Poker Odds

Use the deck of cards class from the notebook we worked through outside of class to write a _Monte Carlo_ code that plays a lot of hands of straight poker (like 100,000).  Count how many of these hands has a particular poker hand (like 3-of-a-kind).  The ratio of # of hands with 3-of-a-kind to total hands is an approximation to the odds of getting a 3-of-a-kind in poker.

You'll want to copy-paste those classes into a `.py` file to allow you to import and reuse them here

(I didn't do that, so I copied and pasted those classes below).

In [3]:
# copied and pasted this cell from the other notebook for now
import random


class Card(object):
    
    def __init__(self, suit=1, rank=2):
        if suit < 1 or suit > 4:
            print("invalid suit, setting to 1")
            suit = 1
        if rank < 2 or rank > 14:
            print("invalid rank, setting to 2")
            rank = 2
        
        self.suit = suit
        self.rank = rank
        

    def value(self):
        """ we want things order primarily by rank then suit """
        return self.suit + (self.rank-1)*14
    
    # we include this to allow for comparisons with < and > between cards 
    def __lt__(self, other):
        return self.value() < other.value()

    def __unicode__(self):
        suits = [u"\u2660".encode('utf-8'),  # spade
                 u"\u2665".encode('utf-8'),  # heart
                 u"\u2666".encode('utf-8'),  # diamond
                 u"\u2663".encode('utf-8')]  # club
        
        r = str(self.rank)
        if self.rank == 11:
            r = "J"
        elif self.rank == 12:
            r = "Q"
        elif self.rank == 13:
            r = "K"
        elif self.rank == 14:
            r = "A"
                
        return r +':'+suits[self.suit-1]
    
    def __str__(self):
        return self.__unicode__()  #.encode('utf-8')
        
        


class Deck(object):
    """ the deck is a collection of cards """

    def __init__(self):

        self.nsuits = 4
        self.nranks = 13
        self.minrank = 2
        self.maxrank = self.minrank + self.nranks - 1

        self.cards = []

        for rank in range(self.minrank,self.maxrank+1):
            for suit in range(1, self.nsuits+1):
                self.cards.append(Card(rank=rank, suit=suit))

    def shuffle(self):
        random.shuffle(self.cards)

    def get_cards(self, num=1):
        hand = []
        for n in range(num):
            hand.append(self.cards.pop())

        return hand
    
    def __str__(self):
        string = ""
        for c in self.cards:
            string += str(c) + " "
        return string

I've added lots of comments, but the basic idea is this:

We are counting any deck of cards that has exactly 3 cards of the same rank. We will investigate 100,000 different hands (5 random cards, chosen randomly from a full deck of 52 cards). For each iteration,

- Generate a new deck of cards, shuffle it, and draw a hand.
- We will have, at most, 5 unique ranks. Create a dictionary, whose keys will be the set of unique ranks in the deck, and whose values will be the number of cards with that rank.
    - For example, if the deck had ranks `5, 9, 8, 4, 9`, the dictionary would be `{5: 1, 9: 2, 8: 1, 4: 1}`.
- For each card in the deck:
    - Get its rank.
    - Check if that rank is already a key in the dictionary.
        - If it's not, add that rank as a new key, and initialize its value to 1.
        - If it is, add 1 to the value of that key.
- Get a list of the values in the dictionary. If one of the values is `3`, then there are 3 cards with the same rank. Add one to the counter that's keeping track of how many hands have 3-of-a-kind.

In [4]:
def n_of_a_kind(nkind=3, ncards=5, N=100000):
    """Determine how often a random hand of cards will contain nkind cards of the same rank.
    Parameters
    ----------
    nkind : int
        The necessary number of cards with the same rank.
    ncards : int
        The number of cards to draw for each hand.
    N : int
        The number of hands to draw.
    Returns
    -------
    ratio : float
        The ratio of the number of hands with nkind cards of the same rank to the total number N."""
    count = 0 # count number of hands w/ n of a kind

    # generate N decks of cards
    for i in range(N):
        # generate the deck, shuffle the cards, and get a new hand:
        deck = Deck()
        deck.shuffle() 
        hand = deck.get_cards(ncards)

        # create an empty dictionary:
        # the keys will be the unique rank(s) of cards in the deck
        # the values will be the number of cards in the hand with that rank
        ranks = {}
        for card in hand:
            if ranks.has_key(card.rank):
                # if the rank is already in the dict, add one to the count for that rank
                ranks[card.rank] += 1
            else:
                # otherwise, this is first card with that rank
                ranks[card.rank] = 1 # initialze value to 1 card with that rank

        # we don't actually care about the specific rank, only the number of cards with the same rank.
        # so, ranks.values() will give a list of the values in the dict (no need for the keys anymore).
        # if this list contains an element that is the same as nkind, then there are nkind cards with same rank.
        # example: here nkind=3, so if any of the values in the dict are 3, we have 3 cards of the same kind.
        if nkind in ranks.values():
            count += 1

    ratio = float(count)/float(N)
    print("Out of {} hands, {} had 3-of-a-kind, or {}%".format(N, count, ratio*100.))
    return ratio

In [5]:
N = 100000 # number of hands
ncards = 5 # number of cards per hand
nkind = 3 # looking for 3 of a kind

# repeat a few times and look at distribution of results
nruns = 10
ratios = []
for i in range(nruns):
    ratios.append(n_of_a_kind(nkind=nkind, ncards=ncards, N=N))

Out of 100000 hands, 2243 had 3-of-a-kind, or 2.243%
Out of 100000 hands, 2272 had 3-of-a-kind, or 2.272%
Out of 100000 hands, 2240 had 3-of-a-kind, or 2.24%
Out of 100000 hands, 2194 had 3-of-a-kind, or 2.194%
Out of 100000 hands, 2291 had 3-of-a-kind, or 2.291%
Out of 100000 hands, 2329 had 3-of-a-kind, or 2.329%
Out of 100000 hands, 2340 had 3-of-a-kind, or 2.34%
Out of 100000 hands, 2217 had 3-of-a-kind, or 2.217%
Out of 100000 hands, 2319 had 3-of-a-kind, or 2.319%
Out of 100000 hands, 2297 had 3-of-a-kind, or 2.297%


In [6]:
avg_ratio = sum(ratios) / float(nruns)
print("Average chance of getting 3-of-a-kind is {:1.2f}%.".format(avg_ratio*100.))

Average chance of getting 3-of-a-kind is 2.27%.
